In [146]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
import glob
import os
from os import walk
from os import path
from bs4 import BeautifulSoup
import requests
from pandas.api.types import CategoricalDtype
import varname
import math
from pandas import Index
from itertools import cycle

os.chdir('C:\Świetlik\KU Leuven\MT Masters thesis\Data')
mypath =os.getcwd()

# pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)

# Functions

In [165]:
# Displaying all rows of the object
def show_all(output):
    option_value = pd.get_option('display.max_rows')
    pd.set_option('display.max_rows', None)
    try:
        display(output)
    finally:
        pd.set_option('display.max_rows', option_value)
        
def show_wide(output):
    option_value = pd.get_option('display.width')
    pd.set_option('display.width', None)
    try:
        display(output)
    finally:
        pd.set_option('display.width', option_value)
        
# No scientific notation
def show_nosci(output):
    option_value = pd.get_option('display.float_format')
    pd.set_option('display.float_format', lambda x: '%.0f' % x)
    try:
        display(output)
    finally:
        pd.set_option('display.float_format', option_value)
        
# turn 
def cm2inch(*tupl):
    inch = 2.54
    if isinstance(tupl[0], tuple):
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)

def interact(data, list1, list2):
    col_to_zip = []
    for i in list1:
        for j in list2:
            col_to_zip.append([i,j])
    inter_col=[]
    for i,j in col_to_zip:
        data[i+'_'+j] = data[i].mul(data[j])
        inter_col.append(i+'_'+j)
    return data[inter_col]

def w_bins(lower_bound, width, max_val):
    ceiling = math.ceil((max_val-lower_bound) / width)*width+1
    return list(range(lower_bound, ceiling, width))

# Category Files (_kategorie_)


In [ ]:
# Scan for files
# all_kat_paths = glob.glob('BBGD/20*/*kate*.csv')

### loading 2004, special treatment

In [ ]:
k04a = pd.read_csv('BBGD/2004/B4X_2004.csv')
k04a.drop('g1', axis=1, inplace=True)
k04a.rok_p = 2004

k04b =  pd.read_csv('BBGD/2004/br2004_01a_1.csv')
k04b = k04b[['nrg','klm', 'trb','woj']]
k04 = k04a.merge(k04b, left_on='nrg', right_on='nrg')

keys = 'rok_p', 'nrg', 'waga', 'los', 'dochg', 'wydg', 'dgr', 'wtukg', 'wp', 'trb',
values = 'rok', 'nrg', 'mn', 'los', 'doch', 'wyd', 'dgr', 'wtuk', 'wp', 'typr',
rename_04_col = dict(zip(keys, values))
k04.rename(columns=rename_04_col, inplace=True)
# k04.loc[k04.typr == 12, 'typr'] = 13
# k04.loc[k04.typr == 11, 'typr'] = 12

### loading other years

In [ ]:
k06 = pd.read_csv('BBGD\\2006\\br2006_kategorie.csv')
k08 = pd.read_csv('BBGD\\2008\\br2008_kategorie.csv')
k10 = pd.read_csv('BBGD\\2010\\br2010_kategorie.csv')
k12 = pd.read_csv('BBGD\\2012\\br2012_kategorie.csv')
k14 = pd.read_csv('BBGD\\2014\\br2014_kategorie.csv')
k16 = pd.read_csv('BBGD\\2016\\br2016_kategorie.csv')
k18 = pd.read_csv('BBGD\\2018\\br2018_kategorie.csv')

# List of data frames
kats = [k04, k06,k08,k10,k12,k14,k16,k18]

## Changes to structure of variables

In [ ]:
# Systemic differences
to_drop = ['region', 'makroreg', 'mn_br04']
to_rename = {'nrg_lp': 'nrg' }
to_order = ['panel', 'gz', 'mn2011']

for i in range(len(kats)):
    kats[i].columns = [x.lower() for x in kats[i].columns]
#     aggregating type category
#     kats[i].loc[kats[i].typr.isin([8,9,10,11]), 'typr'] = 14
    for d in to_drop:
        if d in kats[i].columns:
            kats[i] = kats[i].drop(d, axis=1)
    for r in to_rename:
        if r in kats[i].columns:
            kats[i].rename(columns=to_rename, inplace=True)
    for o in to_order:
        if o in kats[i].columns:
            order = list(kats[i].columns)
            order.remove(o)
            kats[i] = kats[i][order+[o]]

# Unique differences in years
zut_update = {'g_zut': 'g_zut_18', 'd_zut': 'd_zut_18'}
k18.rename(columns=zut_update, inplace=True)

# Find discrepancy in number of categories in categorical variables

In [201]:
# cut off for number of categorical values
cutoff=30

cat_var_for_analysis = []

# Make placeholder data frame
columns = set([i for k in kats for i in k.columns.to_list()])
index = [k.rok[0] for k in kats]
collector = pd.DataFrame(index=index, columns=columns)

# loop over the data
for k in kats:
    y = k.rok[0]
    for col in k:
        v = k[col].unique()
        if len(v) < cutoff:
            collector.loc[y, col] = len(v)
            if col not in cat_var_for_analysis: cat_var_for_analysis.append(col)
        else:
#             define the non categorical variable label
            collector.loc[y, col] = 'X'
cat_var_for_analysis.remove('rok')
collector.fillna('')
# cat_var_for_analysis

,dgr,los,losz,mn2011,wtuk,mc,doch,d_zut,panel,pgrs,wp,gz,rok,woj,mn,rgdn,g_zut,nrg,grs,pgdn,ddp,klm,typr,wyd
2004,X,X,,,X,,X,,,,X,,1,16,X,,,X,,,,6,12,X
2006,X,15,X,,X,12,X,13,,7,X,,1,16,X,X,12,X,5,X,X,6,13,X
2008,X,15,16,,X,12,X,13,2,7,X,,1,16,X,X,12,X,5,X,X,6,13,X
2010,X,13,X,,X,12,X,13,2,7,X,3,1,16,X,X,12,X,5,X,X,6,13,X
2012,X,14,X,X,X,12,X,13,2,7,X,3,1,16,X,X,12,X,5,X,X,6,13,X
2014,X,14,X,,X,12,X,13,2,7,X,3,1,16,X,X,12,X,5,X,X,6,13,X
2016,X,12,X,,X,12,X,13,2,7,X,3,1,16,X,X,12,X,5,X,X,6,13,X
2018,X,13,X,,X,12,X,11,2,7,X,3,1,16,X,X,10,X,5,X,X,6,13,X


## Compare frequences of categoical variables

In [184]:
# cut off for number of categorical values
cutoff=30

# Make multiindex
list_of_cat=[]
for var in cat_var_for_analysis:
    all_cat_in_var = []
    for k in kats:
        if var in k:
            if len(k[var].unique()) < cutoff:
                for x in k[var].unique():
                    if x not in all_cat_in_var:
                        all_cat_in_var.append(x)
    all_cat_in_var.sort()
    list_of_cat.append(all_cat_in_var)

multiindex_list = []
for i in range(len(list_of_cat)):
    multiindex_list.extend(list(zip(cycle([cat_var_for_analysis[i]]), list_of_cat[i])))
multiindex = pd.MultiIndex.from_tuples(multiindex_list, sortorder=True)

# Make Placeholder dataframe
columns = [k.rok[0] for k in kats]
collector = pd.DataFrame(index=multiindex, columns=columns)

# count valus at specific categories
for k in kats:
    y=k.rok[0]
    for var in cat_var_for_analysis:
        if var in k:
            if len(k[var].unique()) < cutoff:
                for cat in k[var].unique():
#                     print(k.loc[k[var]==cat, 'rok'].count())
                    collector.loc[(var, cat), y] = k.loc[k[var]==cat, 'rok'].count()
show_all(collector)

2004   2006   2008   2010   2012   2014   2016   2018
klm   1    4600   4559   4627   4702   4768   4831   4762   4580
      2    3502   3573   3424   3432   3455   3409   3135   3038
      3    2945   2854   2805   2590   2719   2726   2904   2914
      4    6431   6765   6716   6343   6447   6491   6347   6335
      5    4228   3945   4047   4474   4296   4142   4133   3883
      6   10508  15812  15739  15871  15742  15616  15605  15416
typr  1    5816   7150   7830   8473   8701   8942   9453   9808
      2    4122   4378   4316   4189   4068   4100   3725   3174
      3    4361   4618   4160   4128   4067   3910   3558   3169
      4    1473   1619   1319   1200   1090   1023    819    726
      5     628    611    488    433    362    333    192    152
      6    1055    946    821    697    725    746    698    624
      7      86     89     62     67     60     71     66     46
      8    2896   4273   3934   3724   3435   2989   3264   3441
      9     385   1059    970    946    906    892    842    756
      10    218     82     84     77     70     76     76     65
      11   5008    590    526    472    439    375    353    253
      12   6166   6049   6579   6700   7104   7595   7590   7919
      13    NaN   6044   6269   6306   6400   6163   6250   6033
woj   2    2670   2955   2944   2950   2961   2984   2899   2859
      4    1695   1984   1951   1976   1949   1946   1968   1897
      6    1812   2246   2233   2186   2175   2076   2062   2050
      8     826    999   1011    963    966   1011    999    977
      10   2447   2706   2744   2680   2611   2531   2515   2467
      12   2495   3070   3042   3181   3124   3097   3117   3098
      14   4478   5180   5299   5388   5530   5610   5535   5352
      16    912   1056   1046    966   1016   1013    973    918
      18   1541   1988   1955   1956   1927   1870   1878   1896
      20   1007   1210   1212   1204   1220   1157   1152   1164
      22   1799   2044   2025   2032   2141   2196   2230   2173
      24   4244   4606   4464   4504   4444   4348   4297   4114
      26   1011   1289   1241   1285   1230   1214   1193   1185
      28   1170   1401   1414   1381   1451   1493   1478   1456
      30   2680   3143   3141   3143   3109   3116   3066   3085
      32   1427   1631   1636   1617   1573   1553   1524   1475
mc    1     NaN   3115   3113   3106   3130   3123   3081   3035
      2     NaN   3155   3129   3112   3132   3093   3093   3032
      3     NaN   3127   3128   3136   3114   3116   3086   3039
      4     NaN   3117   3105   3102   3105   3095   3067   2997
      5     NaN   3124   3109   3100   3111   3102   3059   3005
      6     NaN   3132   3105   3139   3128   3091   3086   3027
      7     NaN   3109   3092   3104   3094   3087   3090   2975
      8     NaN   3115   3112   3113   3117   3092   3061   2999
      9     NaN   3131   3124   3123   3122   3105   3055   3041
      10    NaN   3112   3103   3116   3125   3097   3059   2997
      11    NaN   3131   3106   3127   3122   3099   3079   3013
      12    NaN   3140   3132   3134   3127   3115   3070   3006
g_zut 1     NaN   9878  10568   9810   9489   9277   9028   8410
      2     NaN   7628   8096   8631   8772   8869   8849   8807
      3     NaN   2276   2002   1909   1841   1689   1689   1555
      4     NaN   2331   2478   2657   2589   2488   2500   2467
      5     NaN      3      5      2      2      4      3     54
      6     NaN     34     32     45     54     40     42  11726
      7     NaN   9794  10144  10435  10580  10622  11009   1931
      8     NaN   3526   2690   2502   2561   2545   2314    788
      9     NaN    611    272    256    246    273    181    358
      10    NaN    726    510    529    593    680    770     70
      11    NaN    641    509    580    622    675    433    NaN
      12    NaN     60     52     56     78     53     68    NaN
d_zut 1     NaN   3285   3133   3050   2796   2712   2592   2277
      2     NaN   2984   3499   3442   3711   

# Download harmonized variables dictionary

In [5]:
# Conversion data base for kategore - all yeras

Convbase_kat = pd.read_excel('200711 Wybór zmiennych.xlsx','klucze do kategorii')

# Index start & end of variable
csi = Convbase_kat[Convbase_kat.code.notna()].index.values
csi = np.append(csi, len(Convbase_kat))
csi = list(zip(csi[:-1]+1,csi[1:]-1))

# Creating dictionaries from values and descriptions
dictionaries=[]
for i ,j in csi:
    keys = Convbase_kat.loc[i:j, 'value']
    values =  Convbase_kat.loc[i:j, 'description']
    convdict = dict(zip(keys, values))
    dictionaries.append(convdict)
    
# Joining dictionaries with labels
conv_kat = pd.DataFrame(Convbase_kat[['code', 'short_name', 'full_name']].dropna().set_index('code'))
conv_kat['dict']=dictionaries

# Codes and short names dictionary
kats_name_dict = dict(zip(conv_kat.index, conv_kat.short_name))

show_wide(conv_kat)

,short_name,full_name,dict
code,,,
woj,województwo,WOJ\nWojewództwo,"{2.0: 'dolnośląskie', 4.0: 'kujawsko-pomorskie', 6.0: 'lubelskie', 8.0: 'lubuskie', 10.0: 'łódzkie', 12.0: 'małopolskie', 14.0: 'mazowieckie', 16.0: 'opolskie', 18.0: 'podkarpackie', 20.0: 'podlas..."
klm,klasa_miejscowości,KLM\nKlasa miejscowości zamieszkania,"{1.0: '500 tys. mieszkańców i więcej', 2.0: '200 - 499 tys. mieszkańców', 3.0: '100 - 199 tys. mieszkańców', 4.0: '20 - 99 tys. mieszkańców', 5.0: 'poniżej 20 tys. mieszkańców', 6.0: 'wieś'}"
gz,stopień_urb,GZ Stopień urbanizacji kraju,"{1.0: 'obszar gęsto zaludniony', 2.0: 'obszar średnio zaludniony', 3.0: 'obszar słabo zaludniony'}"
typr,struktura_gd,TYPR\nTyp biologiczny gospodarstwa domowego,"{1.0: 'małżeństwo bez dzieci', 2.0: 'małżeństwo z 1 dzieckiem na utrzymaniu', 3.0: 'małżeństwo z 2 dzieci na utrzymaniu', 4.0: 'małżeństwo z 3 dzieci na utrzymaniu', 5.0: 'małżeństwo z 4 ( i więce..."
grs,grupa_spo_eko,GRS\nGrupy społeczno-ekonomiczne,"{1.0: 'pracowników', 2.0: 'rolników', 3.0: 'pracujących na własny rachunek', 4.0: 'emerytów i rencistów', 5.0: 'utrzymujących się z niezarobkowych źródeł'}"
pgrs,podgrupa_spo_eko,PGRS \nPodgrupy społeczno-ekonomiczne,"{1.0: 'pracowników na stanowiskach robotniczych', 2.0: 'pracowników na stanowiskach nierobotniczych', 3.0: 'emerytów', 4.0: 'rencistów', 5.0: 'utrzymujących się ze świadczeń społecznych', 6.0: 'ut..."
g_zut_18,głów_źródło_utrz,"D2G, D2D\nGłówne i dodatkowe źródło utrzymania","{1.0: 'praca najemna na stanowisku robotniczym ', 2.0: 'praca najemna na stanowisku nierobotniczym', 3.0: 'użytkowanie gospodarstwa rolnego', 4.0: 'praca na własny rachunek poza gospodarstwem roln..."
d_zut_18,dodat_źródło_utrz,"D2G, D2D\nGłówne i dodatkowe źródło utrzymania","{1.0: 'praca najemna na stanowisku robotniczym ', 2.0: 'praca najemna na stanowisku nierobotniczym', 3.0: 'użytkowanie gospodarstwa rolnego', 4.0: 'praca na własny rachunek poza gospodarstwem roln..."
g_zut,głów_źródło_utrz,"D2G, D2D\nGłówne i dodatkowe źródło utrzymania","{1.0: 'praca najemna na stanowisku robotniczym ', 2.0: 'praca najemna na stanowisku nierobotniczym', 3.0: 'użytkowanie gospodarstwa rolnego', 4.0: 'praca na własny rachunek poza gospodarstwem roln..."


# Replacing values, renaming columns, changing varibales to categories

In [7]:
kat_var_to_cat = set(['województwo', 'klasa_miejscowości', 'stopień_urb', 'struktura_gd','grupa_spo_eko', 'podgrupa_spo_eko',
                  'głów_źródło_utrz', 'dodat_źródło_utrz', 'okres'])

for i in range(len(kats)):
    k = kats[i]
    for col in k:
        if col in conv_kat.index:
            k[col].replace(conv_kat.loc[col, 'dict'], inplace=True)
    k.rename(columns=kats_name_dict, inplace=True)
    for col in kat_var_to_cat:
        if col in k.columns:
            k[col] = k[col].astype('category')

# Overview of variables

In [8]:
all_var = set()
for i in range(len(kats)):
    all_var.update( set(kats[i].columns))
all_var

collector = []
for i in range(len(kats)):
    k = kats[i]
    ks_col =[]
    for col in all_var:
        if col in k.columns:
            ks_col.append('X')
        else:
            ks_col.append(' ')
    collector.append(ks_col)

overview = pd.DataFrame(collector, columns = all_var, index = range(2004, 2019, 2)).transpose()

overview

,2004,2006,2008,2010,2012,2014,2016,2018
grupa_spo_eko,,X,X,X,X,X,X,X
dodat_źródło_utrz,,X,X,X,X,X,X,X
województwo,X,X,X,X,X,X,X,X
rok,X,X,X,X,X,X,X,X
mn2011,,,,,X,,,
waga,X,X,X,X,X,X,X,X
wydatki_ogółem,X,X,X,X,X,X,X,X
stopień_urb,,,,X,X,X,X,X
głów_źródło_utrz,,X,X,X,X,X,X,X
okres,,,X,X,X,X,X,X


# 01a_1

## Loading

In [229]:
a04 = pd.read_csv('BBGD\\2004\\br2004_01a_1.csv')
a06 = pd.read_csv('BBGD\\2006\\br2006_01a_1.csv')
a08 = pd.read_csv('BBGD\\2008\\br2008_01a_1.csv')
a10 = pd.read_csv('BBGD\\2010\\br2010_01a_1.csv')
a12 = pd.read_csv('BBGD\\2012\\br2012_01a_1.csv')
a14 = pd.read_csv('BBGD\\2014\\br2014_01a_1.csv')
a16 = pd.read_csv('BBGD\\2016\\br2016_01a_1.csv')
# joining 1b to 2018 data
a18 = pd.read_csv('BBGD\\2018\\br2018_01a_1.csv')
b18 = pd.read_csv('BBGD\\2018\\br2018_01b.csv')
b18.drop(columns=['rok','woj', 'mc'], inplace=True)
a18 = a18.merge(b18, how='left', left_on='NRG_LP', right_on='NRG_LP')

ats = [a04,a06,a08,a10,a12,a14,a16,a18]

year_labels = range(2004, 2019, 2)

## Changing the structure of variables

In [230]:
# _ - variable missing
# X - continuous variable
# N - number of dscrete values

cutoff=30

all_var = set()
for i in range(len(ats)):
    all_var.update(set(ats[i].columns))
all_var = set([v[:-3] for v in all_var])

collector = []
for i in range(len(ats)):
    a = ats[i]
    as_col =[]
    for col in all_var:
        trunk_col = [v[:-3] for v in a.columns]
        if col in trunk_col:
            index = trunk_col.index(col)
            unique = a[a.columns[index]].unique()
            nr_u = len(unique)
            if nr_u < cutoff:
                as_col.append(nr_u)
            else:
                as_col.append('X')
        else:
            as_col.append('_')
    collector.append(as_col)
overview = pd.DataFrame(collector, columns = all_var, index = year_labels).T

show_all(overview)

,2004,2006,2008,2010,2012,2014,2016,2018
,1,1,1,1,1,1,1,1
d6_,_,2,3,X,X,X,X,_
D5_2,_,_,_,_,_,_,_,2
d5_36,_,_,_,_,_,_,5,_
d5_1,_,_,_,2,2,2,2,_
D3_,_,_,_,_,_,_,_,X
d5_,_,_,_,X,2,2,X,_
d5,_,_,_,X,X,X,X,_
d6_2,_,3,_,_,_,_,_,_
n,_,10,10,8,8,10,8,9


### Unique names for variables in every year

In [231]:
for i in range(len(ats)):
    year = year_labels[i]
    ats[i]['rok'] = year
    ats[i].columns = [x.lower()+'_'+str(year)[-2:] for x in ats[i].columns]


# Download harmonized variables dictionary

In [232]:

# Conversion data base for kategore - all yeras
Convbase_at = pd.read_excel('200711 Wybór zmiennych.xlsx','klucze do 01a')

# Index start & end of variable
csi = Convbase_at[Convbase_at.short_name.notna()].index.values
csi = np.append(csi, len(Convbase_at))
csi = list(zip(csi[:-1]+1,csi[1:]-1))

# Creating dictionaries from values and descriptions
dictionaries=[]
for i ,j in csi:
    keys = Convbase_at.loc[i:j, 'value']
    values =  Convbase_at.loc[i:j, 'description']
    convdict = dict(zip(keys, values))
    dictionaries.append(convdict)
    
# Joining dictionaries with labels
conv_at = pd.DataFrame(Convbase_at[list(range(2004, 2019,2))+['short_name', 'full_name']])
conv_at = conv_at.loc[conv_at.short_name.notna()]
conv_at['dict']=dictionaries
conv_at.reset_index(drop=True, inplace=True)

# Year labels sufix to variables
for col in range(2004, 2019,2):
    conv_at[col] = conv_at[col].apply(lambda x: (str(x)+'_'+str(col)[-2:])  if not pd.isnull(x) else np.nan)
    
# Dictionary for renaming values   
zz = conv_at[list(range(2004, 2019,2))].values
nam_keys=[]
nam_values=[]   
for z in range(len(zz)):
    var = zz[z]
    for label in var:
        if pd.notnull(label):
            nam_keys.append(label)
            nam_values.append(conv_at.loc[z, 'short_name'])       
ats_name_dict = dict(zip(nam_keys,nam_values))

show_wide(conv_at)

,2004,2006,2008,2010,2012,2014,2016,2018,short_name,full_name,dict
0,d32_04,d6_1_06,d6_1_08,d5_1_10,d5_1_12,d5_1_14,d5_1_16,r8_18,rodzaj_budynku,rodzaj budynku,"{1.0: 'budynek wielorodzinny', 2.0: 'dom jedno..."
1,d33_04,d6_6_06,NaN,NaN,NaN,NaN,NaN,NaN,rok_budowy,Uproszczony kres wybudowania budynku,"{1.0: '1800, 1946', 2.0: '1946, 1960', 3.0: '1..."
2,NaN,NaN,d6_6_08,d5_5_10,d5_5_12,d5_5_14,d5_5_16,d5_1_18,rok_budowy,Uproszczony kres wybudowania budynku,"{1.0: '1800, 1946', 2.0: '1946, 1960', 3.0: '1..."
3,NaN,d6_8_06,d6_8_08,d5_6_10,d5_6_12,d5_6_14,d5_6_16,d5_2_18,własność,Własność budynku,"{1.0: 'osoby fizycznej', 2.0: 'spółdzielni mie..."
4,d31_04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tytuł_do_mieszkania,Tytuł prawny do mieszkania,"{1.0: 'własność obciążona', 2.0: 'własność nie..."
5,NaN,d6_9_06,NaN,NaN,NaN,NaN,NaN,NaN,tytuł_do_mieszkania,Tytuł prawny do mieszkania,"{1.0: 'własność obciążona', 2.0: 'własność nie..."
6,NaN,NaN,d6_9_08,d5_7_10,NaN,NaN,NaN,NaN,tytuł_do_mieszkania,Tytuł prawny do mieszkania,"{1.0: 'własność obciążona', 2.0: 'własność nie..."
7,NaN,NaN,NaN,NaN,d5_7_12,d5_7_14,d5_7_16,d5_3_18,tytuł_do_mieszkania,Tytuł prawny do mieszkania,"{1.0: 'własność obciążona', 2.0: 'własność nie..."
8,d344_04,d6_33_06,d6_33_08,d5_30_10,d5_30_12,d5_30_14,d5_21_16,d5_17_18,sieć_ciepłownicza,Czy mieszkanie wyposażone jest w ciepłą wodę?,"{1.0: 'tak, z sieci', 2.0: 'tak, ogrzewaną lok..."
9,d345_04,d6_41_06,d6_41_08,d5_38_10,d5_38_12,d5_38_14,d5_23_16,d5_19_18,sieć_gazowa,Czy mieszkanie wyposażone jest w gaz?,"{1.0: 'tak, z sieci', 2.0: 'tak, z butli', 3.0..."


# Replacing values, renaming columns, changing varibales to categories

In [235]:
at_var_to_cat = set(['rodzaj_budynku', 'rok_budowy', 'własność', 'tytuł_do_mieszkania', 'tytuł_do_mieszkania',
                 'tytuł_do_mieszkania', 'sieć_ciepłownicza', 'sieć_gazowa', 'okres'])

for i in range(len(ats)):
    a = ats[i]
    for col in a.columns:
        if conv_at[year_labels[i]].isin([col]).any():
            index = Index(conv_at[year_labels[i]]).get_loc(col)
            dictionary = conv_at.loc[index, 'dict']
            a[col].replace(dictionary, inplace=True) # this is ok 
    a.rename(columns=ats_name_dict, inplace=True)
    for cc in at_var_to_cat:
        if cc in a.columns:
            a[cc] = a[cc].astype('category')

# Overview of variables

In [236]:
selec_var = set(conv_at.short_name.tolist())
collector = []
for i in range(len(ats)):
    a = ats[i]
    as_col =[]
    for col in selec_var:
        if col in a.columns:
            as_col.append('X')
        else:
            as_col.append(' ')
    collector.append(as_col)
overview = pd.DataFrame(collector, columns = selec_var, index = year_labels).transpose()
overview

,2004,2006,2008,2010,2012,2014,2016,2018
sieć_gazowa,X,X,X,X,X,X,X,X
piętro,,X,X,X,X,X,,
liczba_pięter,,X,X,X,X,X,,
powierzchnia_zajm,X,X,X,X,X,X,X,X
powierzchnia_użytkowa,X,X,X,X,X,X,X,X
gd_id,X,X,X,X,X,X,X,X
okres,X,X,,,,,,
pokoje,X,X,X,X,X,X,X,X
sieć_ciepłownicza,X,X,X,X,X,X,X,X
rodzaj_budynku,X,X,X,X,X,X,X,X


# Combining the data sets

In [ ]:
kiv = kats_important_variables = pd.Index(['gd_id','rok', 'województwo', 'nrg', 'waga', 'głów_źródło_utrz','klasa_miejscowości', 'grupa_spo_eko', 'struktura_gd',
                            'liczba_osób', 'ilczba_osób_żyw','przychody_netto','dochód_rozporządzany', 'wydatki_ogółem', 'okres', 'stopień_urb'])

aiv = ats_important_variables = pd.Index(['sieć_ciepłownicza', 'rok_budowy', 'sieć_gazowa', 'gd_id', 'liczba_pięter', 'okres', 'piętro', 'pokoje',
                           'powierzchnia_użytkowa','powierzchnia_zajm', 'rodzaj_budynku', 'tytuł_do_mieszkania', 'własność'])
bbgd= []
for i in range(len(kats)):
    ikiv = kats[i].columns.intersection(kiv)
    iaiv = ats[i].columns.intersection(aiv)
    bbgd.append(kats[i][ikiv].merge(ats[i][iaiv], how='left', left_on='gd_id', right_on='gd_id'))
bbgd[0].head(5)

# Saving preprocessed data

In [15]:
for i in range(len(bbgd)):
    bbgd[i].to_csv('BBGD/preprocessed_data/BBGD_'+str(year_labels[i])+'.csv', index=False)

# Loading preprocessed data

In [16]:
year_labels
bb_var_to_cat = pd.Index(['sieć_ciepłownicza', 'rok_budowy', 'dodat_źródło_utrz', 'sieć_gazowa', 'grupa_spo_eko', 'głów_źródło_utrz',
                                'klasa_miejscowości', 'okres', 'podgrupa_spo_eko', 'rodzaj_budynku', 'stopień_urb', 'struktura_gd',
                                'tytuł_do_mieszkania', 'województwo', 'własność'])
bbgd=[]
for i in year_labels:
    bb = pd.read_csv('BBGD/preprocessed_data/BBGD_'+str(i)+'.csv')
    ibb_var_to_cat = bb.columns.intersection(bb_var_to_cat)
    bb[ibb_var_to_cat] = bb[ibb_var_to_cat].astype('category')
    bbgd.append(bb)

In [ ]:
for i in bbgd:
    display(i.head(5))